In [1]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string


import warnings
warnings.filterwarnings("ignore")

Переменные, которые можно менять:

In [44]:
risk_horizon = 18
strong = 5 #Вес у переменных с strong predicitive power, ниже будет инструкция к ней

## Пункт 1 ( загрузил для горизонта риска в 18 месяцев, т.к. для 12 нет дефолта)

In [2]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

payments_table = pd.read_sql_query("SELECT * FROM payments", cnx)
payments_table.keys()

Index(['id', 'contract_number', 'date', 'amount_due', 'amount_paid'], dtype='object')

In [3]:
def get_default_with_horizon_risk(contract_number, risk_horizon):
    table = payments_table[payments_table['contract_number'] == contract_number]
    if len(table) >= risk_horizon:
        table = table[:risk_horizon] #функция аналогична просрочке 90+, просто выбираем нужное количество строк как :risk_horizon 
        table = table.reset_index()
        for i in  range(0, len(table['amount_paid'])):
            try:
                if (table['amount_paid'][i] < table['amount_due'][i] and table['amount_paid'][i+1] < table['amount_due'][i+1] and
                table['amount_paid'][i+2] < table['amount_due'][i+2] and table['amount_paid'][i+3] < table['amount_due'][i+3]):
                    date_90 = table['date'][i + 3]
                    date_90 = (str(pd.to_datetime(date_90).year) + '-' + str(pd.to_datetime(date_90).month) + 
                               '-' + str(pd.to_datetime(date_90).day))
                    break

                else:
                    date_90 = '0000-00-00'
            except:
                True
    else:
        date_90 = 'срок кредита меньше горизонта риска'
    return date_90

In [4]:
risk_horizon = 18

data_with_risk_horizon = []
for i in payments_table['contract_number'].unique():
    if get_default_with_horizon_risk(i, risk_horizon) != 'срок кредита меньше горизонта риска':
        if get_default_with_horizon_risk(i, risk_horizon) == '0000-00-00':
            data_with_risk_horizon.append((i, get_default_with_horizon_risk(i, risk_horizon), 0))
        else:
            data_with_risk_horizon.append((i, get_default_with_horizon_risk(i, risk_horizon), 1))

In [5]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS contracts_risk_horizon_{0}_months 
(contract_number INT, 
bankruptcy BOOLEAN, 
date_of_default DATE)""".format(risk_horizon))

cnx.commit()
cnx.close()

In [6]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

for i in data_with_risk_horizon:
    cursor.execute("""INSERT INTO contracts_risk_horizon_{0}_months 
    VALUES
    (%d, %d, '%s')""".format(risk_horizon) % (
    i[0],
    i[2],
    i[1]))
    
cnx.commit()
cnx.close()
## Если перезапускать эту ячейку дважды, то он будет вставлять лишине пеерменные, нужно удалить эту таблицу из БД перед 
## Повторным Запуском

In [7]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS
borrowers_default_with_risk_horizon_{0}_months AS (select A.id, bankruptcy, date_of_default
FROM
(SELECT bankruptcy, id, date_of_default
FROM contracts
INNER JOIN contracts_risk_horizon_{1}_months ON
contracts.contract_number = contracts_risk_horizon_{2}_months.contract_number) AS A
INNER JOIN applications1 ON
applications1.id = A.id)""".format(risk_horizon, risk_horizon, risk_horizon))

cnx.commit()
cnx.close()

In [8]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

applications_with_risk_horizon_18 = pd.read_sql_query("""SELECT bankruptcy, age, age_of_car, children, family, 
gender, marital_status_num, employed_by_num, education_num, position_num, income_type_num, housing_num, 
house_ownership_num, amount, term_month FROM
(SELECT bankruptcy, id, amount, term_month
FROM contracts
INNER JOIN contracts_risk_horizon_{0}_months ON
contracts.contract_number = contracts_risk_horizon_{1}_months.contract_number) AS A
INNER JOIN applications1 ON
applications1.id = A.id""".format(risk_horizon, risk_horizon), cnx)

In [9]:
applications_with_risk_horizon_18

,bankruptcy,age,age_of_car,children,family,gender,marital_status_num,employed_by_num,education_num,position_num,income_type_num,housing_num,house_ownership_num,amount,term_month
0,0,50,NaN,0,2,1,1,14,0,3,3,0,0,1293502,39
1,0,54,26.0,0,1,0,4,5,3,5,4,0,2,135000,23
2,0,55,NaN,0,1,0,4,13,3,3,4,0,2,513000,26
3,0,50,NaN,0,2,0,1,12,3,5,3,0,2,490495,21
4,0,40,17.0,1,3,1,1,1,0,0,0,0,2,1560726,41
5,0,53,8.0,0,2,0,1,12,0,6,3,0,2,1530000,39
6,0,58,NaN,0,2,1,1,25,3,8,2,0,2,1019610,33
7,0,40,NaN,0,1,0,4,3,3,5,4,0,2,405000,23
8,0,31,NaN,1,3,1,1,10,0,3,4,0,2,652500,34
9,0,43,23.0,1,3,0,1,17,3,4,4,0,0,918468,35


## Пункт 2

In [10]:
max_bin = 10
force_bin = 4

# define a binning function
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def woe_iv_categorical(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            conv = char_bin(target, df1[i])
            conv["VAR_NAME"] = i
            count = count + 1
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [11]:
applications_with_risk_horizon_18.keys()

Index(['bankruptcy', 'age', 'age_of_car', 'children', 'family', 'gender',
       'marital_status_num', 'employed_by_num', 'education_num',
       'position_num', 'income_type_num', 'housing_num', 'house_ownership_num',
       'amount', 'term_month'],
      dtype='object')

#### Выберем колонки с числовыми и категориальными переменными

In [12]:
num_columns = ['age', 'age_of_car', 'amount', 'term_month']

In [13]:
cat_columns = ['gender', 'marital_status_num', 'education_num', 'income_type_num', 'house_ownership_num', 'children']

#employed_by_num, position_num убираю, потому что там слишком много категорий ,а без большого 
#количества данных адекватно их сгруппировать не представляется возможным

#family убираем, потому что о них информация заложена в переменных семейное положение и детии

#housing убираем, слишком смещённые данные в ользу одной категории

In [14]:
df_cat = applications_with_risk_horizon_18[cat_columns].copy()

In [15]:
df_cat.head(5)

,gender,marital_status_num,education_num,income_type_num,house_ownership_num,children
0,1,1,0,3,0,0
1,0,4,3,4,2,0
2,0,4,3,4,2,0
3,0,1,3,3,2,0
4,1,1,0,0,2,1


In [16]:
df_num = applications_with_risk_horizon_18[num_columns].copy()

In [17]:
df_num.head(5)

,age,age_of_car,amount,term_month
0,50,NaN,1293502,39
1,54,26.0,135000,23
2,55,NaN,513000,26
3,50,NaN,490495,21
4,40,17.0,1560726,41


#### Присваивать значения буду на основании woe

In [18]:
woe_iv_cat = woe_iv_categorical(df_cat, applications_with_risk_horizon_18.bankruptcy)[0]
woe_iv_cat

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,gender,0,0,21,0,0.000000,21,1.000000,0.0,0.477273,0.000000,0.309605
1,gender,1,1,24,1,0.041667,23,0.958333,1.0,0.522727,0.648695,0.309605
2,marital_status_num,0,0,4,0,0.000000,4,1.000000,0.0,0.090909,0.000000,2.950541
3,marital_status_num,1,1,28,0,0.000000,28,1.000000,0.0,0.636364,0.000000,2.950541
4,marital_status_num,3,3,1,0,0.000000,1,1.000000,0.0,0.022727,0.000000,2.950541
5,marital_status_num,4,4,9,0,0.000000,9,1.000000,0.0,0.204545,0.000000,2.950541
6,marital_status_num,5,5,3,1,0.333333,2,0.666667,1.0,0.045455,3.091042,2.950541
7,education_num,0,0,14,0,0.000000,14,1.000000,0.0,0.318182,0.000000,0.142123
8,education_num,1,1,1,0,0.000000,1,1.000000,0.0,0.022727,0.000000,0.142123
9,education_num,3,3,30,1,0.033333,29,0.966667,1.0,0.659091,0.416894,0.142123


#### Для примера я присвою максимум по три значения в каждой колонке, показав как бы я это делал, будь у нас больше данных, потому что считаю абсолютно неочевидным, ставить эти баллы в каждой категории на глаз

*Максимальное значение для каждой колонки - 10*

In [19]:
df_cat['gender_score'] = [10 if x  == 1 else 2 for x in df_cat['gender']]

In [20]:
df_cat['marital_status_score'] = np.where(df_cat['marital_status_num'] == 5, 10, 
                                          np.where(df_cat['marital_status_num'] == 3, 6, 2))

In [21]:
df_cat['education_score'] = np.where(df_cat['education_num'] == 0, 2, 
                                          np.where(df_cat['education_num'] == 1, 6, 10))

In [22]:
df_cat['income_type_score'] = np.where(df_cat['income_type_num'] == 2, 6, 
                                          np.where(df_cat['income_type_num'] == 4, 8, 4))

In [23]:
df_cat['house_ownership_score'] = np.where(df_cat['house_ownership_num'] == 2, 8, 
                                          np.where(df_cat['house_ownership_num'] == 0, 4, 10))

In [24]:
df_cat['children_score'] = np.where(df_cat['children'] == 2, 6, 
                                          np.where(df_cat['children'] == 0, 8, 4))

In [25]:
iv_categorical = woe_iv_cat[['VAR_NAME', 'IV']].drop_duplicates().copy()
iv_categorical = iv_categorical.reset_index()
iv_categorical = iv_categorical.drop(['index'], axis = 1)
iv_categorical

,VAR_NAME,IV
0,gender,0.309605
1,marital_status_num,2.950541
2,education_num,0.142123
3,income_type_num,0.244113
4,house_ownership_num,0.058598
5,children,0.086851


#### Тоже самое с числовыми

In [26]:
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)


def woe_iv_cont(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [27]:
woe_iv_num = woe_iv_cont(df_num, applications_with_risk_horizon_18.bankruptcy)[0]
woe_iv_num

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,21.0,39.0,15,0.0,0.000000,15.0,1.000000,0.0,0.340909,0.000000,0.859010
1,age,40.0,52.0,16,0.0,0.000000,16.0,1.000000,0.0,0.363636,0.000000,0.859010
2,age,53.0,67.0,14,1.0,0.071429,13.0,0.928571,1.0,0.295455,1.219240,0.859010
3,age_of_car,2.0,3.0,4,0.0,0.000000,4.0,1.000000,0.0,0.090909,0.000000,0.142123
4,age_of_car,NaN,NaN,0,0.0,NaN,0.0,NaN,0.0,0.000000,NaN,0.142123
5,age_of_car,7.0,7.0,1,0.0,0.000000,1.0,1.000000,0.0,0.022727,0.000000,0.142123
6,age_of_car,8.0,8.0,2,0.0,0.000000,2.0,1.000000,0.0,0.045455,0.000000,0.142123
7,age_of_car,10.0,10.0,2,0.0,0.000000,2.0,1.000000,0.0,0.045455,0.000000,0.142123
8,age_of_car,13.0,13.0,1,0.0,0.000000,1.0,1.000000,0.0,0.022727,0.000000,0.142123
9,age_of_car,14.0,14.0,1,0.0,0.000000,1.0,1.000000,0.0,0.022727,0.000000,0.142123


In [28]:
df_num['age_score'] = np.where( (df_num['age'] <= 39) & (df_num['age'] >= 21), 4, 
                               np.where ((df_num['age'] <= 52) & (df_num['age'] >= 40), 8, 10))

In [29]:
df_num['age_of_car_score'] = np.where( (df_num['age_of_car'] <= 8) & (df_num['age_of_car'] >= 2), 4, 
                               np.where ((df_num['age_of_car'] <= 14) & (df_num['age_of_car'] >= 9), 6, 10))

In [30]:
df_num['amount_score'] = np.where((df_num['amount'] <= 454500), 4, 
                               np.where ((df_num['amount'] <= 773680.0) & (df_num['amount'] >= 454501), 6, 10))

In [31]:
df_num['term_month_score'] = np.where( (df_num['term_month'] <= 23), 4, 
                               np.where ((df_num['term_month'] <= 30) & (df_num['term_month'] >= 24), 6, 10))

In [32]:
iv_num = woe_iv_num[['VAR_NAME', 'IV']].drop_duplicates().copy()
iv_num = iv_num.reset_index()
iv_num = iv_num.drop(['index'], axis = 1)
iv_num

,VAR_NAME,IV
0,age,0.859010
1,age_of_car,0.142123
2,amount,0.780772
3,term_month,0.780772


In [33]:
IV_table = iv_categorical.append(iv_num).sort_values('IV')

In [34]:
IV_table = IV_table.reset_index().drop(['index'], axis = 1)

Отбор переменных по IV:
 + <0.02 бесполезный
 + [0.02, 0.1] слабый
 + [0.1, 0.3] обычный
 + [0.3, 0.5] сильный
 + больше 0.5 подозрительно

In [35]:
IV_table['predicive power'] = np.where((IV_table.IV < 0.02), 'useless',
                                       np.where((IV_table.IV >= 0.02) & (IV_table.IV <= 0.1), 'weak',
                                               np.where((IV_table.IV > 0.1) & (IV_table.IV <= 0.3), 'medium',
                                                       np.where((IV_table.IV > 0.3) & (IV_table.IV <= 0.5), 'strong', 'suspicious'))))
                                       

## Пункт 3

#### Зададим веса на основании деления по IV

In [36]:
IV_table

,VAR_NAME,IV,predicive power
0,house_ownership_num,0.058598,weak
1,children,0.086851,weak
2,education_num,0.142123,medium
3,age_of_car,0.142123,medium
4,income_type_num,0.244113,medium
5,gender,0.309605,strong
6,amount,0.780772,suspicious
7,term_month,0.780772,suspicious
8,age,0.859010,suspicious
9,marital_status_num,2.950541,suspicious


##### Предоставим пользовтелю задать вес для переменной с strong predictive power, остальные веса подсчитаются автоматически, при этом medium и weak являются релевантными, useless слабый вес - остальное на suspicious

In [37]:
strong = 5
# Не меньше чем 1.82, иначе нарушится монотонность весов, то есть сильная переменная будет иметь меньший вес, чем средняя
# И не больше 10
if strong < 1.82 or strong > 10:
    print('Ошибка, некорректные значения для переменной strong')

# ________________________________________________________________________________________________

In [38]:
medium = (10 - strong)/1.5/3
weak = (10 - strong - 3 * medium)/2
useless = (10-strong - 3 * medium  - weak)/3
suspicious = (10-strong - 3 *  medium - weak - useless)/4

In [39]:
print('medium=', medium, '\n', 'weak=', weak, '\n',  'useless=', useless, '\n',  'suspicious=', suspicious)

medium= 1.1111111111111112 
 weak= 0.8333333333333333 
 useless= 0.27777777777777773 
 suspicious= 0.1388888888888889


In [40]:
table_with_scores = {}
table_with_scores = pd.concat([df_cat, df_num], axis = 1)
table_with_scores['bankruptcy'] = applications_with_risk_horizon_18.bankruptcy
table_with_scores.head(5)

,gender,marital_status_num,education_num,income_type_num,house_ownership_num,children,gender_score,marital_status_score,education_score,income_type_score,...,children_score,age,age_of_car,amount,term_month,age_score,age_of_car_score,amount_score,term_month_score,bankruptcy
0,1,1,0,3,0,0,10,2,2,4,...,8,50,NaN,1293502,39,8,10,10,10,0
1,0,4,3,4,2,0,2,2,10,8,...,8,54,26.0,135000,23,10,10,4,4,0
2,0,4,3,4,2,0,2,2,10,8,...,8,55,NaN,513000,26,10,10,6,6,0
3,0,1,3,3,2,0,2,2,10,4,...,8,50,NaN,490495,21,8,10,6,4,0
4,1,1,0,0,2,1,10,2,2,4,...,4,40,17.0,1560726,41,8,10,10,10,0


In [41]:
df = table_with_scores.copy()

In [42]:
table_with_scores['score'] = (df.children_score * useless + df.house_ownership_score * weak + df.education_score * medium +
                              df.age_of_car_score * medium + df.income_type_score * medium + df.gender_score * strong + 
                              df.amount_score * suspicious + df.term_month_score * suspicious + df.age_score * suspicious + 
                              df.marital_status_score * suspicious)

In [43]:
table_with_scores.score.describe()

count    45.000000
mean     67.567901
std      21.936704
min      31.666667
25%      48.055556
50%      78.055556
75%      88.611111
max      95.555556
Name: score, dtype: float64